In [9]:
#this section saves the capacity and capacity addition from the ESM and aggregates them for the specified locations
#works for all SCMs
from zen_garden.postprocess.results import Results
import pandas as pd
from pathlib import Path

# Configuration
dataset_name = "esm_pv_wind" #adjust dataset_name for your dataset
base_path = Path(r".\outputs")
results_path = base_path / dataset_name
output_dir = Path("./csv_output") / dataset_name

selected_technologies = {"heat_pump", "photovoltaics", "wind_offshore", "wind_onshore"} #adjust relevant selected technologies
specific_locations = {"DE", "CH"} #specify locations that are not summed into ROE (rest of europe)

# Create output directory if it doesn't exist
output_dir.mkdir(parents=True, exist_ok=True)

# Load Results
res = Results(results_path)

capacity = res.get_full_ts("capacity").reset_index()
capacity_addition = res.get_full_ts("capacity_addition").reset_index()

# Filter Technologies
capacity = capacity[capacity["technology"].isin(selected_technologies)]
capacity_addition = capacity_addition[capacity_addition["technology"].isin(selected_technologies)]

# Save Filtered CSVs
capacity.round(4).to_csv(output_dir / "capacity.csv", index=False)
capacity_addition.round(4).to_csv(output_dir / "capacity_addition.csv", index=False)

# Aggregation Setup
all_locations = set(capacity["location"].unique())
roe_locations = all_locations - specific_locations

def aggregate_by_location(df, location_set, location_name):
    return (
        df[df["location"].isin(location_set)]
        .groupby(["technology", "capacity_type"])
        .sum(numeric_only=True)
        .assign(location=location_name)
    )

# Aggregate Data
capacity_aggregated = pd.concat([
    aggregate_by_location(capacity, {"DE"}, "DE"),
    aggregate_by_location(capacity, {"CH"}, "CH"),
    aggregate_by_location(capacity, roe_locations, "ROE")
]).reset_index()

capacity_addition_aggregated = pd.concat([
    aggregate_by_location(capacity_addition, {"DE"}, "DE"),
    aggregate_by_location(capacity_addition, {"CH"}, "CH"),
    aggregate_by_location(capacity_addition, roe_locations, "ROE")
]).reset_index()

# Save Aggregated CSVs
capacity_aggregated.round(4).to_csv(output_dir / "capacity_aggregated_by_location.csv", index=False)
capacity_addition_aggregated.round(4).to_csv(output_dir / "capacity_addition_aggregated_by_location.csv", index=False)

print(f"CSV files for dataset '{dataset_name}' saved in: {output_dir}")


CSV files for dataset 'esm_pv_wind' saved in: csv_output\esm_pv_wind


In [10]:
# rename_year_columns.py
# This script updates aggregated CSV column headers with actual years from system configuration.
# Assumes dataset_name, output_dir, and results_path are already defined.

import json
import pandas as pd

# Load system.json
system_path = results_path / "system.json"
with open(system_path, "r") as f:
    system_config = json.load(f)

reference_year = system_config["reference_year"]
interval = system_config["interval_between_years"]
optimized_years = system_config["optimized_years"]

# Create rename mapping
year_labels = [str(reference_year + i * interval) for i in range(optimized_years)]
rename_columns = {str(i): year_labels[i] for i in range(len(year_labels))}

# Function to rename year columns in a CSV
def rename_year_columns(csv_path):
    df = pd.read_csv(csv_path)
    df = df.rename(columns=rename_columns)
    df.to_csv(csv_path, index=False)
    print(f"Renamed year columns in: {csv_path.name}")

# Apply to aggregated CSV files
rename_year_columns(output_dir / "capacity_aggregated_by_location.csv")
rename_year_columns(output_dir / "capacity_addition_aggregated_by_location.csv")

Renamed year columns in: capacity_aggregated_by_location.csv
Renamed year columns in: capacity_addition_aggregated_by_location.csv


In [11]:
#specific for wind (write values to demand_yearly_variation.csv)
#import pandas as pd
import shutil
#from pathlib import Path
#import json

# CONFIGURATION
#dataset_name = "energy_transition_ref_2021"

# Base paths
#base_path = Path(r".\outputs")
#output_dir = Path("./CSV_output") / dataset_name
esm_capacity_path = output_dir / "capacity_addition_aggregated_by_location.csv"

original_dir = Path(r"C:\Users\nicol\OneDrive\Dokumente\ZEN-garden_SP\Data_Wind\Data_WT")
new_dir = Path(r"C:\Users\nicol\OneDrive\Dokumente\ZEN-garden_SP\Data_Wind\Data_WT_new")
original_dyv_path = original_dir / "set_carriers" / "Turbine" / "demand_yearly_variation.csv"
target_dyv_path = new_dir / "set_carriers" / "Turbine" / "demand_yearly_variation.csv"

# --- Step 1: Clone the original directory ---
if new_dir.exists():
    print(f"Directory already exists: {new_dir}")
else:
    shutil.copytree(original_dir, new_dir)
    print(f"Created clone of wind model input: {new_dir}")

# --- Step 2: Load year labels from system.json ---
system_path = base_path.parent / dataset_name / "system.json"
with open(system_path, "r") as f:
    system_config = json.load(f)

reference_year = system_config["reference_year"]
interval = system_config["interval_between_years"]
optimized_years = system_config["optimized_years"]
year_labels = [str(reference_year + i * interval) for i in range(optimized_years)]

# --- Step 3: Load original demand_yearly_variation to preserve all nodes ---
df_existing = pd.read_csv(original_dyv_path)

# --- Step 4: Prepare updated wind data from ESM ---
country_map = {
    "DE": "DEU",
    "CH": "CHE",
    "AT": "AUT",
    "NL": "NLD",
    "ROE": "ROE",
    "ROW": "ROW",
}

df = pd.read_csv(esm_capacity_path)
df_wind = df[df["technology"].isin(["wind_onshore", "wind_offshore"])]
df_grouped = df_wind.groupby(["location"]).sum(numeric_only=True).reset_index()
df_grouped["node"] = df_grouped["location"].map(country_map)
df_grouped = df_grouped[df_grouped["node"].notna()].drop(columns=["location"])

# Transpose to match demand_yearly_variation structure
df_new = df_grouped.set_index("node").T
df_new.index.name = "year"
df_new = df_new.reset_index()

# Fix year column using actual labels from system.json
df_new["year"] = year_labels[:len(df_new)]
df_new["year"] = df_new["year"].astype(str)
df_existing["year"] = df_existing["year"].astype(str)

# Set year as index for merging
df_new = df_new.set_index("year")
df_existing = df_existing.set_index("year")

# --- Step 5: Merge values only for matching columns ---
updated_cols = [col for col in df_new.columns if col in df_existing.columns]

for col in updated_cols:
    df_existing[col] = df_new[col].combine_first(df_existing[col])

# --- Step 6: Save final file ---
df_result = df_existing.reset_index()
df_result.to_csv(target_dyv_path, index=False)
print(f"Final demand_yearly_variation.csv written to:\n{target_dyv_path}")


Directory already exists: C:\Users\nicol\OneDrive\Dokumente\ZEN-garden_SP\Data_Wind\Data_WT_new
Final demand_yearly_variation.csv written to:
C:\Users\nicol\OneDrive\Dokumente\ZEN-garden_SP\Data_Wind\Data_WT_new\set_carriers\Turbine\demand_yearly_variation.csv
